In [1]:
header=["outlook","temperature","humidity","wind","decision"]

training_data=[
['sunny','hot','high','weak','no'],
['sunny','hot','high','strong','no'],
['overcast','hot','high','weak','yes'],
['rain','mild','high','weak','yes'],
['rain','cool','normal','weak','yes'],
['rain','cool','normal','strong','no'],
['overcast','cool','normal','strong','yes'],
['sunny','mild','high','weak','no'],
#['sunny','cool','normal','weak','yes'],
['rain','mild','normal','weak','yes'],
['sunny','mild','normal','strong','yes'],
['overcast','mild','high','strong','yes'],
['overcast','hot','normal','weak','yes'],
#['rain','mild','high','strong','no'],
]

In [2]:
def unique_values(data,col):
    return set([row[col] for row in data])

unique_values(training_data,0)

{'overcast', 'rain', 'sunny'}

In [3]:
def class_counts(data):
    counts={}
    for row in data:
        label = row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

class_counts(training_data)

{'no': 4, 'yes': 8}

In [4]:
class Question:
    def __init__(self,column,value): 
        self.column=column
        self.value=value
    def match(self,example): 
        val = example[self.column]
        return val == self.value 
    def __repr__(self):
        return "Is %s %s %s?"%(header[self.column],"==",str(self.value))

In [5]:
def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [6]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [7]:
def info_gain(left,right,uncertainity):
    p = float(len(left)) / (len(left) + len(right))
    q = float(len(right)) / (len(left) + len(right))
    return uncertainity - p*gini(left) - q*gini(right)

In [8]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0])-1
    for col in range(n_features):
        values = unique_values(rows,col)
        for val in values:
            question = Question(col,val)
            true_rows,false_rows = partition(rows,question)
            
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            
            gain = info_gain(true_rows,false_rows,current_uncertainity)
            
            if gain >= best_gain:
                best_gain,best_question = gain,question
    return best_gain,best_question

In [9]:
class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)
        
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [10]:
def build_tree(rows):
    gain,question = find_best_split(rows)
    if gain==0:
        return Leaf(rows)
    true_rows,false_rows = partition(rows,question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question,true_branch,false_branch)

In [11]:
def print_tree(node,spacing=" "):
    if isinstance(node, Leaf):
        print(spacing+"Predict",node.predictions)
        return
    print(spacing + str(node.question))
    
    print(spacing + '-->True:')
    print_tree(node.true_branch,spacing+"    ")
    
    print(spacing + '-->False:')
    print_tree(node.false_branch,spacing+"    ")

In [12]:
def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total * 100)) + "%"
    return probs

In [13]:
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

In [14]:
mytree = build_tree(training_data)

In [15]:
print_tree(mytree)

 Is outlook == sunny?
 -->True:
     Is humidity == normal?
     -->True:
         Predict {'yes': 1}
     -->False:
         Predict {'no': 3}
 -->False:
     Is wind == strong?
     -->True:
         Is outlook == rain?
         -->True:
             Predict {'no': 1}
         -->False:
             Predict {'yes': 2}
     -->False:
         Predict {'yes': 5}


In [16]:
testing_data = [
    ['rain','mild','high','strong','no'],
    ['sunny','cool','normal','weak','yes'],
]

In [17]:
for row in testing_data:
        print("Actual: %s. Predicted: %s."%(row[-1],print_leaf(classify(row,mytree))))

Actual: no. Predicted: {'no': '100%'}.
Actual: yes. Predicted: {'yes': '100%'}.
